In [ ]:
! pip install evaluate

In [1]:
# https://huggingface.co/docs/transformers/training

# Fine tune a pre-trained model using GPU

### Prepare a dataset

Load <a href="https://huggingface.co/datasets/yelp_review_full">Yelp Review</a> data from huggingface 

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

### Load Tokenizer to process text

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

#### Take a subset of data to test the complete  pipeline first

In [4]:
# taking subset of data to reduce time for this tryout run
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Train
### Using PyTorch Trainer
Load the model first, we are using bert-base-cased. 

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Add training hyperparams

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Trainer will not help evaulting the model, we need to pass function for evalution. Here we are using evaulate.

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Set evaluation strategy to epoch to track evaluation metrics every epoch during fine-tuning

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Fine-tune the model by calling trainer.train()

In [13]:
trainer.train()

/tmp/pip_packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.191229,0.473000
2,No log,0.998391,0.590000
3,No log,1.036741,0.610000


TrainOutput(global_step=375, training_loss=0.9105623372395834, metrics={'train_runtime': 352.2727, 'train_samples_per_second': 8.516, 'train_steps_per_second': 1.065, 'total_flos': 789354427392000.0, 'train_loss': 0.9105623372395834, 'epoch': 3.0})